### 3.1.1 Setup
---

In [1]:
%cd /root/rollout-strategies/

/root/rollout-strategies


In [7]:
import boto3
import pandas as pd
import sagemaker
from sagemaker import get_execution_role, s3, estimator

#pd.set_option('display.max_columns', 500)     # Make sure we can see all of the columns
#pd.set_option('display.max_rows', 10)         # Keep the output on one page

BUCKET = 'talk-20200720'
PREFIX = 'data'
LOCAL_DATA_DIRECTORY = f'./data'
print(f"Artifacts will be written to s3://{BUCKET}/{PREFIX}")

# Session variables we'll use throughout the notebook
sm_session = sagemaker.Session()
boto_session = sm_session.boto_session
sagemaker_client = boto_session.client('sagemaker')
role = get_execution_role()
print(f'Role ARN: {role}')

Artifacts will be written to s3://talk-20200720/data
Role ARN: arn:aws:iam::209970524256:role/service-role/AmazonSageMaker-ExecutionRole-20200618T144956


### 3.1.2 Data
---

To see how the dataset was preprocessed, see this notebook: [XGBoost customer churn notebook that starts with the original dataset](https://github.com/awslabs/amazon-sagemaker-examples/blob/master/introduction_to_applying_machine_learning/xgboost_customer_churn/xgboost_customer_churn.ipynb). 

In [3]:
data = pd.read_csv(f'{LOCAL_DATA_DIRECTORY}/training-dataset-with-header.csv')

data.head()

,Churn,Account Length,VMail Message,Day Mins,Day Calls,Eve Mins,Eve Calls,Night Mins,Night Calls,Intl Mins,...,State_WI,State_WV,State_WY,Area Code_408,Area Code_415,Area Code_510,Int'l Plan_no,Int'l Plan_yes,VMail Plan_no,VMail Plan_yes
0,0,106,0,274.4,120,198.6,82,160.8,62,6.0,...,0,0,0,0,0,1,1,0,1,0
1,0,28,0,187.8,94,248.6,86,208.8,124,10.6,...,0,0,1,0,1,0,1,0,1,0
2,1,148,0,279.3,104,201.6,87,280.8,99,7.9,...,0,0,0,0,1,0,1,0,1,0
3,0,132,0,191.9,107,206.9,127,272.0,88,12.6,...,0,0,0,0,0,1,1,0,1,0
4,0,92,29,155.4,110,188.5,104,254.9,118,8.0,...,0,0,0,0,0,1,1,0,0,1


In [4]:
s3_input_train = sm_session.upload_data(f'{LOCAL_DATA_DIRECTORY}/train.csv',
                                               bucket=BUCKET,
                                               key_prefix=PREFIX)

s3_input_validation = sm_session.upload_data(f'{LOCAL_DATA_DIRECTORY}/validation.csv',
                                                    bucket=BUCKET,
                                                    key_prefix=PREFIX)

s3_input_train = sagemaker.s3_input(s3_data=s3_input_train, content_type='csv')
s3_input_validation = sagemaker.s3_input(s3_data=s3_input_validation, content_type='csv')

'upload_data' method will be deprecated in favor of 'S3Uploader' class (https://sagemaker.readthedocs.io/en/stable/s3.html#sagemaker.s3.S3Uploader) in SageMaker Python SDK v2.
'upload_data' method will be deprecated in favor of 'S3Uploader' class (https://sagemaker.readthedocs.io/en/stable/s3.html#sagemaker.s3.S3Uploader) in SageMaker Python SDK v2.
's3_input' class will be renamed to 'TrainingInput' in SageMaker Python SDK v2.
's3_input' class will be renamed to 'TrainingInput' in SageMaker Python SDK v2.


### 3.1.3 Training
---

In [5]:
from sagemaker.amazon.amazon_estimator import get_image_uri

xgboost_image_name = get_image_uri(boto_session.region_name, 'xgboost', repo_version='0.90-2')
xgboost_image_name

'get_image_uri' method will be deprecated in favor of 'ImageURIProvider' class in SageMaker Python SDK v2.
	get_image_uri(region, 'xgboost', '1.0-1').


'257758044811.dkr.ecr.us-east-2.amazonaws.com/sagemaker-xgboost:0.90-2-cpu-py3'

### Train Model A

In [8]:
model_A = estimator.Estimator(image_name=xgboost_image_name,
                                role=role,
                                train_instance_count=1,
                                train_instance_type='ml.m4.xlarge',
                                output_path=f"s3://{BUCKET}/model-A",
                                base_job_name="model-A",
                                sagemaker_session=sm_session)

model_A.set_hyperparameters(max_depth=5,
                          subsample=0.8,
                          num_round=600,
                          eta=0.2,
                          gamma=4,
                          min_child_weight=6,
                          silent=0,
                          objective='binary:logistic')

model_A.fit({'train': s3_input_train,
               'validation': s3_input_validation})

2020-07-04 13:30:36 Starting - Starting the training job...
2020-07-04 13:30:38 Starting - Launching requested ML instances......
2020-07-04 13:31:40 Starting - Preparing the instances for training...
2020-07-04 13:32:31 Downloading - Downloading input data...
2020-07-04 13:32:48 Training - Downloading the training image...
2020-07-04 13:33:29 Training - Training image download completed. Training in progress.INFO:sagemaker-containers:Imported framework sagemaker_xgboost_container.training
INFO:sagemaker-containers:Failed to parse hyperparameter objective value binary:logistic to Json.
Returning the value itself
INFO:sagemaker-containers:No GPUs detected (normal if no gpus installed)
INFO:sagemaker_xgboost_container.training:Running XGBoost Sagemaker in algorithm mode
INFO:root:Determined delimiter of CSV input is ','
INFO:root:Determined delimiter of CSV input is ','
INFO:root:Determined delimiter of CSV input is ','
[13:33:31] 2333x69 matrix with 160977 entries loaded from /opt/ml/in

### Deploy Model A

In [11]:
endpoint_name = 'churn-model'
model_name = 'model-A'

model_A_predictor = model_A.deploy(initial_instance_count=1,
                                   instance_type="ml.m4.xlarge",
                                   endpoint_name=endpoint_name,
                                   model_name=model_name)

-------------!

### View the EndpointConfig

In [17]:
endpoint_config = sagemaker_client.describe_endpoint(EndpointName=model_A_predictor.endpoint)

In [18]:
import pprint
pprint.pprint(endpoint_config)

{'CreationTime': datetime.datetime(2020, 7, 4, 14, 0, 35, 501000, tzinfo=tzlocal()),
 'EndpointArn': 'arn:aws:sagemaker:us-east-2:209970524256:endpoint/churn-model',
 'EndpointConfigName': 'churn-model',
 'EndpointName': 'churn-model',
 'EndpointStatus': 'InService',
 'LastModifiedTime': datetime.datetime(2020, 7, 4, 14, 6, 44, 760000, tzinfo=tzlocal()),
 'ProductionVariants': [{'CurrentInstanceCount': 1,
                         'CurrentWeight': 1.0,
                         'DeployedImages': [{'ResolutionTime': datetime.datetime(2020, 7, 4, 14, 0, 37, 505000, tzinfo=tzlocal()),
                                             'ResolvedImage': '257758044811.dkr.ecr.us-east-2.amazonaws.com/sagemaker-xgboost@sha256:3c733d0214998ee69e8cf53864c666ae238e3b360bfcd453f9321fbf9d953536',
                                             'SpecifiedImage': '257758044811.dkr.ecr.us-east-2.amazonaws.com/sagemaker-xgboost:0.90-2-cpu-py3'}],
                         'DesiredInstanceCount': 1,
               

In [19]:
endpoint_config['ProductionVariants']

[{'VariantName': 'AllTraffic',
  'DeployedImages': [{'SpecifiedImage': '257758044811.dkr.ecr.us-east-2.amazonaws.com/sagemaker-xgboost:0.90-2-cpu-py3',
    'ResolvedImage': '257758044811.dkr.ecr.us-east-2.amazonaws.com/sagemaker-xgboost@sha256:3c733d0214998ee69e8cf53864c666ae238e3b360bfcd453f9321fbf9d953536',
    'ResolutionTime': datetime.datetime(2020, 7, 4, 14, 0, 37, 505000, tzinfo=tzlocal())}],
  'CurrentWeight': 1.0,
  'DesiredWeight': 1.0,
  'CurrentInstanceCount': 1,
  'DesiredInstanceCount': 1}]

## Blue/Green Deployment

![blue green deployment](./figures/blue-green.png)

[Figure Source](https://d1.awsstatic.com/whitepapers/architecture/wellarchitected-Machine-Learning-Lens.pdf)

### Train Model B

In [12]:
model_B = estimator.Estimator(image_name=xgboost_image_name,
                                role=role,
                                train_instance_count=1,
                                train_instance_type='ml.m4.xlarge',
                                output_path=f"s3://{BUCKET}/model-B",
                                base_job_name="model-B",
                                sagemaker_session=sm_session)

model_B.set_hyperparameters(max_depth=3,
                          subsample=0.5,
                          num_round=600,
                          eta=0.2,
                          gamma=4,
                          min_child_weight=6,
                          silent=0,
                          objective='binary:logistic')

model_B.fit({'train': s3_input_train,
               'validation': s3_input_validation})

2020-07-04 14:07:48 Starting - Starting the training job...
2020-07-04 14:07:50 Starting - Launching requested ML instances......
2020-07-04 14:08:52 Starting - Preparing the instances for training...
2020-07-04 14:09:43 Downloading - Downloading input data...
2020-07-04 14:10:00 Training - Downloading the training image...
2020-07-04 14:10:46 Uploading - Uploading generated training model.INFO:sagemaker-containers:Imported framework sagemaker_xgboost_container.training
INFO:sagemaker-containers:Failed to parse hyperparameter objective value binary:logistic to Json.
Returning the value itself
INFO:sagemaker-containers:No GPUs detected (normal if no gpus installed)
INFO:sagemaker_xgboost_container.training:Running XGBoost Sagemaker in algorithm mode
INFO:root:Determined delimiter of CSV input is ','
INFO:root:Determined delimiter of CSV input is ','
INFO:root:Determined delimiter of CSV input is ','
[14:10:42] 2333x69 matrix with 160977 entries loaded from /opt/ml/input/data/train?forma

### Create Model Object

In [25]:
endpoint_name = 'model-B-endpoint'
model_name = 'model-B'

model_B_predictor = model_B.deploy(initial_instance_count=1,
                                   instance_type="ml.m4.xlarge",
                                   endpoint_name=endpoint_name,
                                   model_name=model_name)

---------------!

#### Step 1. Create a new endpoint configuration, using the same production variants for the existing live model and for the new model.

In [27]:
variant_A = {'ModelName': 'model-A',
            'VariantName': 'blue',
            'InitialInstanceCount': 1,
            'InstanceType': 'ml.t2.medium',
            'InitialVariantWeight': 1.0}

variant_B = {'ModelName': 'model-B',
            'VariantName': 'green',
            'InitialInstanceCount': 1,
            'InstanceType': 'ml.t2.medium',
            'InitialVariantWeight': 0.0}

In [28]:
sagemaker_client.create_endpoint_config(EndpointConfigName = 'blue-green',
                                        ProductionVariants = [variant_A, variant_B])

{'EndpointConfigArn': 'arn:aws:sagemaker:us-east-2:209970524256:endpoint-config/blue-green',
 'ResponseMetadata': {'RequestId': 'd3e70eaf-5de1-428e-be54-340a51f37092',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'd3e70eaf-5de1-428e-be54-340a51f37092',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '91',
   'date': 'Sat, 04 Jul 2020 14:48:18 GMT'},
  'RetryAttempts': 0}}

#### Step 2. Update the existing live endpoint with the new endpoint configuration. Amazon SageMaker creates the required infrastructure for the new production variant and updates the weights without any downtime.

In [29]:
sagemaker_client.update_endpoint(EndpointName='churn-model', EndpointConfigName='blue-green')

{'EndpointArn': 'arn:aws:sagemaker:us-east-2:209970524256:endpoint/churn-model',
 'ResponseMetadata': {'RequestId': '2fbdb826-592c-41f2-9c59-310f5f24f529',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '2fbdb826-592c-41f2-9c59-310f5f24f529',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '79',
   'date': 'Sat, 04 Jul 2020 14:50:43 GMT'},
  'RetryAttempts': 0}}

#### Step 3. Switch traffic to the new model through an API call.

In [30]:
sagemaker_client.update_endpoint_weights_and_capacities(
    EndpointName='churn-model',
    DesiredWeightsAndCapacities=[
        {
            'VariantName': 'blue',
            'DesiredWeight': 0.0
        },
        {
            'VariantName': 'green',
            'DesiredWeight': 1.0
        }
    ]
)

{'EndpointArn': 'arn:aws:sagemaker:us-east-2:209970524256:endpoint/churn-model',
 'ResponseMetadata': {'RequestId': '8db73a04-0210-4bbd-a623-07f6777d76a4',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '8db73a04-0210-4bbd-a623-07f6777d76a4',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '79',
   'date': 'Sat, 04 Jul 2020 15:00:10 GMT'},
  'RetryAttempts': 0}}

#### Step 4. Create a new endpoint configuration with only the new production variant and apply it to the endpoint. 

In [32]:
sagemaker_client.update_endpoint(EndpointName='churn-model', EndpointConfigName='model-B-endpoint')

{'EndpointArn': 'arn:aws:sagemaker:us-east-2:209970524256:endpoint/churn-model',
 'ResponseMetadata': {'RequestId': 'b559901b-b46a-4ce6-9839-fe1cec7d6ca6',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'b559901b-b46a-4ce6-9839-fe1cec7d6ca6',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '79',
   'date': 'Sat, 04 Jul 2020 15:02:58 GMT'},
  'RetryAttempts': 0}}